In [ ]:
import pandas as pd
import numpy as np
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split
# !pip install fasttext
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,roc_auc_score
import seaborn as sns

In [ ]:
df=pd.read_csv("data_sentiment.csv")

In [ ]:
df.head()

,Unnamed: 0,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,Category,Pos,Neg,Neu,compound,sentiment
0,47021,A1LZ63OMHD6YNK,B000HDNCHI,Old Man River,"[1, 1]",chage tire 1200 mile cotiet harshel gatorski ...,5.0,These Are Good Tires,1378944000,"09 12, 2013",Sports & Outdoors,0.000,0.000,1.000,0.0000,Neutral
1,273341,A3KVXLRJ2O6RAS,B007D4R4ZK,Old Broad,"[0, 0]","i'v bee usig prouct sever year ow couter oly,...",5.0,Price,1366329600,"04 19, 2013",Health & Personal Care,0.149,0.000,0.851,0.6369,Very Satisfied
2,227768,A3NGLZL8GFGZOE,B004H468S8,Brian,"[0, 0]",o't give prouct 5 star ratig o't use featur me...,4.0,Omron BP760 7 Blood Pressure Monitor,1343001600,"07 23, 2012",Health & Personal Care,0.000,0.128,0.872,-0.2960,Dissatisfied
3,194040,A2RHFO4FX45UVU,B0038QQ7TG,teambarrett14,"[0, 0]",favourit wash eve husba uless make tropic trai...,5.0,the best! besides homemade,1325721600,"01 5, 2012",Health & Personal Care,0.000,0.000,1.000,0.0000,Neutral
4,159231,A2VH0UT5EQFB6P,B0028K3IZY,Loveguitar,"[0, 0]","eoorat smell ice whe first applie, time goe sc...",3.0,Very Strong Scent,1243468800,"05 28, 2009",Health & Personal Care,0.088,0.000,0.912,0.8478,Very Satisfied


In [ ]:
df.loc[df["compound"]>.6,"sentiment"]=5
df.loc[(df["compound"]<=.6)&(df["compound"]>.2),"sentiment"]=4
df.loc[(df["compound"]<=.2)&(df["compound"]>-.2),"sentiment"]=3
df.loc[(df["compound"]<=-.2)&(df["compound"]>-.6),"sentiment"]=2
df.loc[df["compound"]<-.6,"sentiment"]=1

In [ ]:
df.sentiment.unique()

array([3, 5, 2, 4, 1], dtype=object)

In [ ]:
df["sentiment"]=df["sentiment"].astype("int")

# Creating model

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
import re
stemmer = nltk.SnowballStemmer("english")
stopword=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\AR\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AR\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def clean(text):
    text = str(text).lower()
    text = re.sub('[.*?]', '', text)
    text = re.sub('https?://S+|www.S+', '', text)
    text = re.sub('n', '', text)
    text = re.sub('w*dw*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
df["reviewText"] = df["reviewText"].apply(clean)

In [ ]:
df=df[["reviewText","sentiment"]]

## Splitting data into test and train

In [ ]:
X = df.reviewText
y = df.sentiment
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

## Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=5000)
x_train = tfidf_vectorizer.fit_transform(X_train)
x_test = tfidf_vectorizer.transform(X_test)

lr = LogisticRegression(max_iter=200, verbose=1)
ovr = OneVsRestClassifier(lr)

# fit the model on the training data
ovr.fit(x_train, y_train)

# predictions for test set
y_pred = ovr.predict(x_test)
# print(multilabel_binarizer.inverse_transform(y_pred)[7])

# compute F1-score
f1_score1 = f1_score(y_test, y_pred, average="micro")
print("Micro F1-score for logistic regression: "+ str(f1_score1))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Micro F1-score for logistic regression: 0.7796794772055392


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.2s finished


In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.58      0.12      0.20       926
           2       0.41      0.18      0.25      2653
           3       0.67      0.69      0.68      7268
           4       0.68      0.53      0.59     11509
           5       0.85      0.98      0.91     29060

    accuracy                           0.78     51416
   macro avg       0.64      0.50      0.53     51416
weighted avg       0.76      0.78      0.76     51416



# Multinomial NB

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=5000)
x_train = tfidf_vectorizer.fit_transform(X_train)
x_test = tfidf_vectorizer.transform(X_test)

lr = MultinomialNB()
ovr = OneVsRestClassifier(lr)

# fit the model on the training data
ovr.fit(x_train, y_train)

# predictions for test set
y_pred = ovr.predict(x_test)
# print(multilabel_binarizer.inverse_transform(y_pred)[7])

# compute F1-score
f1_score1 = f1_score(y_test, y_pred, average="micro")
print("Micro F1-score for logistic regression: "+ str(f1_score1))

Micro F1-score for logistic regression: 0.5654076552046056


Prediction will be done using logistic regression since it has a better score as compared to Naiv bias

In [ ]:
df1=pd.read_json("Sports_and_Outdoors_5.json",lines=True)
df2=pd.read_json("Health_and_Personal_Care_5.json",lines=True)
df1["category"]="Sports and outdoors"
df2["category"]="Health and personal care"

In [ ]:
final_df=pd.concat([df1,df2],axis=0)

In [ ]:
final_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,category
0,AIXZKN4ACSKI,1881509818,David Briner,"[0, 0]",This came in on time and I am veru happy with ...,5,Woks very good,1390694400,"01 26, 2014",Sports and outdoors
1,A1L5P841VIO02V,1881509818,Jason A. Kramer,"[1, 1]",I had a factory Glock tool that I was using fo...,5,Works as well as the factory tool,1328140800,"02 2, 2012",Sports and outdoors
2,AB2W04NI4OEAD,1881509818,J. Fernald,"[2, 2]",If you don't have a 3/32 punch or would like t...,4,"It's a punch, that's all.",1330387200,"02 28, 2012",Sports and outdoors
3,A148SVSWKTJKU6,1881509818,"Jusitn A. Watts ""Maverick9614""","[0, 0]",This works no better than any 3/32 punch you w...,4,It's a punch with a Glock logo.,1328400000,"02 5, 2012",Sports and outdoors
4,AAAWJ6LW9WMOO,1881509818,Material Man,"[0, 0]",I purchased this thinking maybe I need a speci...,4,"Ok,tool does what a regular punch does.",1366675200,"04 23, 2013",Sports and outdoors


In [ ]:
final_df["reviewText"] = final_df["reviewText"].apply(clean)

In [ ]:
predict_text=final_df["reviewText"]

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=5000)
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=5000)
x_train = tfidf_vectorizer.fit_transform(X_train)
x_test = tfidf_vectorizer.transform(X_test)
lr = LogisticRegression(max_iter=200, verbose=1)
ovr = OneVsRestClassifier(lr)
ovr.fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.2s finished


OneVsRestClassifier(estimator=LogisticRegression(max_iter=200, verbose=1))

In [ ]:
y_pred = ovr.predict(tfidf_vectorizer.fit_transform(predict_text))

In [ ]:
y_pred=pd.DataFrame(y_pred,columns=["sentiment"])

In [ ]:
y_pred.describe()

,sentiment
count,642692.000000
mean,3.536534
std,0.806235
min,1.000000
25%,3.000000
50%,3.000000
75%,4.000000
max,5.000000


In [ ]:
final_df.reset_index(inplace=True)
predicted_sentiment=pd.concat([final_df,y_pred],axis=1)

In [ ]:
predicted_sentiment.to_csv("predicted_sentiment.csv")

In [ ]:
predicted_sentiment.sentiment.unique()

array([3, 4, 2, 5, 1])